In [24]:
import pandas as pd

def dt_inplace(df):
    """Automatically detect and convert (in place!) each
    dataframe column of datatype 'object' to a datetime just
    when ALL of its non-NaN values can be successfully parsed
    by pd.to_datetime().  Also returns a ref. to df for
    convenient use in an expression.
    """
    from pandas.errors import ParserError
    for c in df.columns[df.dtypes=='object']: #don't cnvt num
        try:
            df[c]=pd.to_datetime(df[c])
        except (ParserError,ValueError): #Can't cnvrt some
            pass # ...so leave whole column as-is unconverted
    return df
def read_csv(*args, **kwargs):
    """Drop-in replacement for Pandas pd.read_csv. It invokes
    pd.read_csv() (passing its arguments) and then auto-
    matically detects and converts each column whose datatype
    is 'object' to a datetime just when ALL of the column's
    non-NaN values can be successfully parsed by
    pd.to_datetime(), and returns the resulting dataframe.
    """
    return dt_inplace(pd.read_csv(*args, **kwargs))


In [27]:
#Before inserting into database, need to verify datatypes. We will create a new csv not rewrite, in order to keep the original.
#note that dt_auto.py located in base directory. That enables the read_csv function without needed pd
from dt_auto import read_csv
df=read_csv('awarding_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135144 entries, 0 to 135143
Data columns (total 48 columns):
 #   Column                                                         Non-Null Count   Dtype         
---  ------                                                         --------------   -----         
 0   contract_transaction_unique_key                                135144 non-null  object        
 1   contract_award_unique_key                                      135144 non-null  object        
 2   award_id_piid                                                  135144 non-null  object        
 3   modification_number                                            135144 non-null  object        
 4   transaction_number                                             128824 non-null  float64       
 5   parent_award_agency_id                                         101838 non-null  object        
 6   parent_award_agency_name                                       101838 non-null  obje

In [38]:
# Fix SyntaxError by removing line breaks in the values
df = df.replace({"\n": " "}, regex=True)

# Get column names and data types from DataFrame
column_names = df.columns.tolist()
dtypes = [str(df[col].dtype) for col in column_names]

# Generate CREATE TABLE query
create_query = "CREATE TABLE new_table (\n"
for name, dtype in zip(column_names, dtypes):
    if dtype == 'int64':  # Check if data type is integer
        dtype = "numeric"  # Adjusted data type for PostgreSQL
    else:
        dtype = "character varying(255) COLLATE pg_catalog.\"default\""  # Adjusted data type for PostgreSQL
    create_query += f"    {name} {dtype},\n"
create_query = create_query.rstrip(',\n')  # Remove the last comma and newline
create_query += "\n);"

# Generate INSERT INTO query
insert_query = f"INSERT INTO new_table ({', '.join(column_names)})\nVALUES "
for row in df.itertuples(index=False):
    values = ', '.join([f"'{str(value).replace("'", "''") if pd.notna(value) and not isinstance(value, int) else str(value)}'" for value in row])
    insert_query += f"\n    ({values}),"
insert_query = insert_query.rstrip(',')  # Remove the last comma
insert_query += ";"

# Write CREATE TABLE query to a file
with open('create_query.sql', 'w') as create_file:
    create_file.write(create_query)

# Write INSERT INTO query to a file
with open('insert_query.sql', 'w') as insert_file:
    insert_file.write(insert_query)

SyntaxError: EOL while scanning string literal (<ipython-input-38-d945d4f31574>, line 22)

In [39]:
#created awarding_data table in db
#column header for "Covid-19", State Department issues were fixed during processing
#TODO: Update this to work with a config file

#the respective create and insert queries are loaded into the following script in place of sql

#note: the create table query worked. The insert query had still had issues with the following:
#leading zeros in decimal integer literals
#error with single quote: NAT'L ENVIR.SATELLITE DATA&INFO, DEFC of '9', 'USS O'KANE  DDG'
#include datatime types in create_query.sql
#fix for Nat dtype
#ERROR:  value too long for type character varying(255


#fields containing values with characters longer than 255
# Iterate over each column in the DataFrame
#Use below to check which problem columns
'''for column in df.columns:
    # Filter values exceeding the varchar(255) limit
    exceeding_values = df[df[column].astype(str).str.len() > 255][column]

    # Print the exceeding values
    if not exceeding_values.empty:
        print(f"Exceeding values in column '{column}':")
        for value in exceeding_values:
            print(value)
        print()'''

#'object_classes_funding_this_award'
#'program_activities_funding_this_award'
#'treasury_accounts_funding_this_award'







'for column in df.columns:\n    # Filter values exceeding the varchar(255) limit\n    exceeding_values = df[df[column].astype(str).str.len() > 255][column]\n\n    # Print the exceeding values\n    if not exceeding_values.empty:\n        print(f"Exceeding values in column \'{column}\':")\n        for value in exceeding_values:\n            print(value)\n        print()'